# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.sparse as sp

from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, jaccard_score

import networkx as nx
# import dgl

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.transforms import NormalizeFeatures,to_undirected
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
import torch_geometric.utils as utils

In [2]:
# Setting up GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Graph Creation

In [3]:
node = pd.read_csv('node_information.csv', header=None)
train = pd.read_csv('train.txt', header=None, names=['Source','Target','Edge'], delim_whitespace=True)
test = pd.read_csv('test.txt', header=None, names=['Source','Target'], delim_whitespace=True)

In [4]:
train 

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1
...,...,...,...
10491,81,6464,0
10492,4,6433,0
10493,3341,6717,1
10494,1562,4656,0


In [5]:
node

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592,7584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3593,7589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,7593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3595,7594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
node.loc[:, node.columns != node.columns[0]] = node.loc[:, node.columns != node.columns[0]].astype(int)
node.head()

,0,1,2,3,4,5,6,7,8,9,...,923,924,925,926,927,928,929,930,931,932
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
nodes = [i for i in node[0]]
nodes[:10]

[0, 4, 5, 6, 7, 9, 10, 11, 13, 17]

In [8]:
train.head()

,Source,Target,Edge
0,939,3809,1
1,2442,5784,1
2,179,3809,1
3,857,2280,1
4,1358,5722,1


In [9]:
# Creates the oriented graph
diG=nx.DiGraph()

# add the list of all nodes
diG.add_nodes_from(nodes)

# adds the corresponding links between two nodes in trainset
for i in range(train.shape[0]):
    if train['Edge'][i] == 1:
        diG.add_edge(train['Source'][i], train['Target'][i])

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  

# Add the attributes to nodes
nx.set_node_attributes(diG, node_attributes, "x")


print("The number of nodes: {}".format(diG.number_of_nodes()))
print("The number of edges: {}".format(diG.number_of_edges()))

The number of nodes: 3597
The number of edges: 5248


In [14]:
# Transform networkx graph to PyG graph
G = utils.from_networkx(diG, group_node_attrs=['x'])
print(G)

Data(edge_index=[2, 5248], x=[3597, 932])


In [15]:
G.edge_index

tensor([[   0,    0,    1,  ..., 3546, 3553, 3590],
        [ 439, 1096, 2949,  ..., 3546, 3572, 3590]])

# Train-Val Split

In [16]:
split = T.RandomLinkSplit(
    num_val=0.2,
    num_test=0,
    is_undirected=False,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0,
)
train_data, val_data, _ = split(G)
print('train_data:', train_data)
print('val_data:', val_data)

train_data: Data(edge_index=[2, 4199], x=[3597, 932], edge_label=[8398], edge_label_index=[2, 8398])
val_data: Data(edge_index=[2, 4199], x=[3597, 932], edge_label=[2098], edge_label_index=[2, 2098])


# GNN

In [19]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [20]:
model = Net(G.x.shape[1], 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=400)

Epoch: 010, Train Loss: 0.684, Val AUC: 0.604
Epoch: 020, Train Loss: 0.638, Val AUC: 0.528
Epoch: 030, Train Loss: 0.591, Val AUC: 0.522
Epoch: 040, Train Loss: 0.553, Val AUC: 0.516
Epoch: 050, Train Loss: 0.529, Val AUC: 0.530
Epoch: 060, Train Loss: 0.514, Val AUC: 0.537
Epoch: 070, Train Loss: 0.500, Val AUC: 0.533
Epoch: 080, Train Loss: 0.490, Val AUC: 0.540
Epoch: 090, Train Loss: 0.482, Val AUC: 0.535
Epoch: 100, Train Loss: 0.478, Val AUC: 0.538
Epoch: 110, Train Loss: 0.476, Val AUC: 0.532
Epoch: 120, Train Loss: 0.471, Val AUC: 0.533
Epoch: 130, Train Loss: 0.469, Val AUC: 0.532
Epoch: 140, Train Loss: 0.463, Val AUC: 0.535
Epoch: 150, Train Loss: 0.464, Val AUC: 0.537
Epoch: 160, Train Loss: 0.461, Val AUC: 0.533
Epoch: 170, Train Loss: 0.455, Val AUC: 0.541
Epoch: 180, Train Loss: 0.455, Val AUC: 0.540
Epoch: 190, Train Loss: 0.456, Val AUC: 0.543
Epoch: 200, Train Loss: 0.453, Val AUC: 0.541
Epoch: 210, Train Loss: 0.449, Val AUC: 0.532
Epoch: 220, Train Loss: 0.452, Val

In [21]:
@torch.no_grad()
def test_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return out, roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

# Make Predictions

In [23]:
new_diG = nx.DiGraph()
new_diG.add_nodes_from(nodes)

# Create a dictionary containing node features
node_attributes = {}
for _, row in node.iterrows():
    node_attributes[row[0]] = torch.from_numpy(np.array(row[1:])).to(torch.float)  


nx.set_node_attributes(new_diG, node_attributes, "x")


new_G = utils.from_networkx(new_diG, group_node_attrs=['x'])
print(new_G)

Data(edge_index=[2, 0], x=[3597, 932])


In [29]:
# Create a dictionary to map node names to indices
node_index = {node: i for i, node in enumerate(new_diG.nodes())}

link_probs = []
model.eval()

for i in range(test.shape[0]):
    # Convert source and target node names to indices
    source_idx = node_index[test['Source'][i]]
    target_idx = node_index[test['Target'][i]]
    
    source = test.iloc[i]['Source']
    target = test.iloc[i]['Target']
    new_diG.add_edge(source, target)
    new_G = utils.from_networkx(new_diG, group_node_attrs=['x'])
    new_G = new_G.to(device)
    
    # Encode the source and target nodes
    z = model.encode(new_G.x, new_G.edge_index)

    # Decode the link probability between the source and target nodes
    link_prob = model.decode(z, torch.tensor([[source_idx], [target_idx]]).to(device)).view(-1).sigmoid()
    link_probs.append(link_prob.cpu().item())

# Print the link probabilities
print(link_probs)

[5.3016581535339355, 9.35700798034668, 11.368773460388184, 0.216136172413826, 1.9118207693099976, 3.5190927982330322, 8.040682792663574, 0.9033364057540894, 3.4738223552703857, 2.101104497909546, 3.171929359436035, 5.428716659545898, 3.327714443206787, -1.4885302782058716, 1.5386723279953003, 3.672747850418091, 12.777795791625977, 3.73968243598938, 0.4549683928489685, 4.673640251159668, 2.3690309524536133, 1.9512485265731812, 9.062685012817383, 1.5073511600494385, 0.5873745679855347, 0.5482859015464783, 1.193981409072876, 1.6865986585617065, 1.9336897134780884, 1.867075800895691, 5.382487773895264, 2.3719916343688965, 2.0655293464660645, 0.6357498168945312, 3.9480013847351074, 1.8281958103179932, 0.35193413496017456, 2.388927459716797, 1.115922451019287, 0.4995853900909424, 2.351179361343384, 1.7141937017440796, 0.4668993353843689, 3.258441209793091, 6.304347038269043, 2.5047402381896973, 0.9683724641799927, 2.3993709087371826, 0.5591705441474915, 1.8959834575653076, 3.8443870544433594

In [26]:
test_predictions = pd.DataFrame(columns = ['Source', 'Target', 'Edge'], dtype=object)
# Iterate through rows
for i in range(len(link_probs)):
    source_val = test['Source'][i] 
    target_val = test['Target'][i]
    edge_val = 1 if link_probs[i] >= 0.5 else 0
    
    # Add the row to the DataFrame
    test_predictions = test_predictions.append({
        'Source': source_val,
        'Target': target_val,
        'Edge': edge_val
    }, ignore_index=True)

C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_13776\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_13776\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_13776\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictions.append({
C:\Users\Davina Chen\AppData\Local\Temp\ipykernel_13776\1709194919.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_predictions = test_predictio

In [27]:
test_predictions[test_predictions['Edge']==1]

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


In [28]:
test_predictions

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


In [157]:
# Join the two DataFrames
preds = pd.merge(test, test_predictions, on=['Source', 'Target'])

In [158]:
preds

,Source,Target,Edge
0,3425,4524,1
1,1620,2617,1
2,4832,6317,1
3,4984,7298,1
4,385,5481,1
...,...,...,...
3493,1548,2957,1
3494,717,1756,1
3495,1731,3976,1
3496,426,1120,1


# Submission

In [159]:
submission = pd.DataFrame()
submission['ID'] = list(range(preds.shape[0]))
submission['Predicted'] = list(preds['Edge'])
submission.to_csv('submission.csv', index = False)